In [183]:
import pandas as pd
import numpy as np
from IPython.display import display

f = {'business':['FX','FX','CR','a'],
     'A/L':['A','L','#','#'],
     'Company':['#','#','#','207'],
    'adj':['a1','a2','a3','b1']}
filter = pd.DataFrame(data=f)


d = {'business': ['FX','FX','CR'],
     'A/L': ['A','A','L'],
     'Amt':[1,2,3],
     'Company': ['207','1','2']}
data = pd.DataFrame(data=d)


# store the original column names
cols = filter.columns
cols1=cols[:-1]


# add a column to store in a (hashable) tuple the columns with no '#'
filter['wild'] = filter.apply(lambda x: tuple(col for col in cols1
                                             if x[col] != '#'), axis=1)

# Ok, we can now use that new filter table to filter the business table
result = pd.concat([data.merge(df, on=k, suffixes=('', '_y'),
                           )[cols]
                for k, df in filter.groupby('wild')]).sort_index()

final_result=pd.merge(data,result,how='left').fillna('-')

display(data,filter,final_result)

,business,A/L,Amt,Company
0,FX,A,1,207
1,FX,A,2,1
2,CR,L,3,2


,business,A/L,Company,adj,wild
0,FX,A,#,a1,"(business, A/L)"
1,FX,L,#,a2,"(business, A/L)"
2,CR,#,#,a3,"(business,)"
3,a,#,207,b1,"(business, Company)"


,business,A/L,Amt,Company,adj
0,FX,A,1,207,a1
1,FX,A,2,1,a1
2,CR,L,3,2,a3


In [184]:
import pandas as pd
import numpy as np
from IPython.display import display

f = {'business':['FX','FX','CR','a'],
     'A/L':['A','L','#','#'],
     'Company':['#','#','#','207'],
    'adj':['a1','a2','a3','b1']}
filter = pd.DataFrame(data=f)


d = {'business': ['FX','FX','CR'],
     'A/L': ['A','A','L'],
     'Amt':[1,2,3],
     'Company': ['207','1','2']}
data = pd.DataFrame(data=d)



# store the original column names
cols = filter.columns
cols1=cols[:-1]

# remove any alternate value if a # is already present:
tosimp = pd.DataFrame({col: filter[col].str.contains('#')&
                       filter[col].str.contains('\|')
                       for col in cols1})

# add a column to store in a (hashable) tuple the columns with no '#'
filter['wild'] = filter.apply(lambda x: tuple(col for col in cols1
                                             if x[col] != '#'), axis=1)

# now explode the fields containing a '|'
tosimp = pd.DataFrame({col: filter[col].str.contains('\|')
                       for col in cols1})

# again, store in a new column the columns containing a '|'
tosimp['wild'] = filter.apply(lambda x: tuple(col for col in cols1
                                             if '|' in filter.loc[x.name, col]),
                              axis=1)

# compute a new filter table with one single value per field (or #)
# by grouping on tosimp['wild']
dfl = [filter[tosimp['wild'].astype(str)=='()']]
for k, df in filter[tosimp['wild'].astype(str)!='()'].groupby(tosimp['wild']):
    for ix, row in df.iterrows():
        tmp = pd.MultiIndex.from_product([df.loc[ix, col].split('|')
                                          for col in k], names=k).to_frame(None)
        l = len(tmp)
        dfl.append(pd.DataFrame({col: tmp[col]
                                 if col in k else [row[col]] * l
                                 for col in cols1}))

filter2 = pd.concat(dfl)

# Ok, we can now use that new filter table to filter the business table
result = pd.concat([data.merge(df, on=k, suffixes=('', '_y'),
                               right_index=True)[cols1]
                    for k, df in filter2.groupby('wild')]).sort_index()

final_result=pd.merge(data,result,how='left').fillna('-')

display(data,filter,final_result)


,business,A/L,Amt,Company
0,FX,A,1,207
1,FX,A,2,1
2,CR,L,3,2


,business,A/L,Company,adj,wild
0,FX,A,#,a1,"(business, A/L)"
1,FX,L,#,a2,"(business, A/L)"
2,CR,#,#,a3,"(business,)"
3,a,#,207,b1,"(business, Company)"


,business,A/L,Amt,Company
0,FX,A,1,207
1,FX,A,2,1
2,CR,L,3,2
